# Import packages

In [ ]:

%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
if Path.cwd().parts[-1] != 'Multifirefly-Project':
    if Path.cwd().parts[-1] != 'notebooks':
        os.chdir('..')
    from add_path import find_path
    current_path = find_path()
    os.chdir(current_path)

from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.neural_analysis_by_topic.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.neural_analysis_by_topic.planning_and_neural import planning_neural_class, planning_neural_utils, planning_neural_helper_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regression_utils2, ml_methods_class, classification_utils, ml_plotting_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, gpfa_regression_utils, plot_gpfa_utils, gpfa_helper_class

import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

Changed the directory to 'Multifirefly-Project'.
Added /Users/dusiyi/Documents/Multifirefly-Project/multiff_analysis/methods to the path.
done
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Retrieve data

## get data

In [8]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"

In [2]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0416"

In [4]:
reduce_y_var_lags = False
planning_data_by_point_exists_ok = True
y_data_exists_ok = True

ref_point_mode='time after cur ff visible'
ref_point_value=0.1
normalize = False
eliminate_outliers = False
use_curv_to_ff_center = False
curv_of_traj_mode = 'distance'
window_for_curv_of_traj=[-25, 25]
truncate_curv_of_traj_by_time_of_capture = True

pn = planning_neural_class.PlanningAndNeural(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)
pn.planning_data_by_point, cols_to_drop = general_utils.drop_columns_with_many_nans(
    pn.planning_data_by_point)
pn.get_x_and_y_data_for_modeling(exists_ok=y_data_exists_ok, reduce_y_var_lags=reduce_y_var_lags)

Loaded binned_spikes_df from all_monkey_data/processed_neural_data/monkey_Schro/data_0416/binned_spikes_df_0p02.csv
Loaded planning_data_by_point from all_monkey_data/planning_and_neural/monkey_Schro/data_0416/planning_data_by_point/opt_arc_stop_closest/test/cur_vis_0_1_window_-50cm_0cm
Loaded planning_data_by_point from all_monkey_data/planning_and_neural/monkey_Schro/data_0416/planning_data_by_point/opt_arc_stop_closest/control/cur_vis_0_1_window_-50cm_0cm
Overlap: [  3073   7127  24450  44992  71018  83166 115592]
Percentage overlap relative to df1: 0.07%
Percentage overlap relative to df2: 0.09%
Average percentage overlap: 0.08%
Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0


/Users/dusiyi/Documents/Multifirefly-Project/multiff_analysis/methods/neural_data_analysis/neural_analysis_by_topic/neural_vs_behavioral/prep_monkey_data.py:57: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  monkey_info_in_bins = monkey_info_in_bins.bfill(


There are 434 duplicated point_index in bin_info. Note: one_behav_idx_per_bin is True
Dropping columns with more than 30.0% missing values ...
0.04% of rows are dropped in planning_data_by_bin due to having missing values
binned_spikes_df.shape: (520849, 87)
self.x_var.shape: (13963, 86)
self.y_var.shape: (13963, 46)
Getting y_var_reduced...
====================Dropping features with high correlation in all columns based on threshold 0.97====================
Processing subset 1 of 1, 46 features in total.
11 columns out of 46 dropped: ['segment_end_time', 'angle_from_m_before_stop_to_nxt_ff', 'time', 'nxt_ff_distance', 'stop_time', 'cur_ff_distance', 'cur_opt_arc_end_heading', 'target_index', 'bin', 'segment_start_time', 'nxt_opt_arc_dheading']

11 out of 46 (23.91%) are dropped after Dropping features with high correlation in subsets of features

Summary: 11 out of 46 (23.91%) are dropped after calling drop_columns_with_high_corr. 
** 35 features are left **

====================Among

# get planning_data by segment

In [ ]:
# make sure that the n_seconds_before_stop is greater than 2

In [10]:
df_sub = pn.planning_data_by_bin[pn.planning_data_by_bin['segment_duration'] > 2].copy()
# print and drop columns with na
columns_before_drop = df_sub.columns
df_sub.dropna(axis=1, inplace=True)
columns_after_drop = df_sub.columns
print(f'Columns dropped dur to Nans: {np.array(set(columns_before_drop) - set(columns_after_drop))}')


Columns dropped dur to Nans: {'nxt_cntr_arc_curv', 'angle_cntr_arc_from_cur_end_to_nxt', 'angle_opt_arc_from_cur_end_to_nxt', 'cur_opt_arc_curv', 'cur_opt_arc_end_heading', 'cur_cntr_arc_end_heading', 'cur_cntr_arc_curv', 'nxt_opt_arc_curv'}


In [ ]:
segment_duration = 2
bin_width = 0.2

# Get unique segments and their stop times
unique_segments = df_sub[['segment', 'stop_time']].drop_duplicates()

df_sub = df_sub.sort_values(by=['segment', 'bin'])
all_seg_subset = []

for _, segment_row in unique_segments.iterrows():
    segment_id = segment_row['segment']
    seg_end_time = segment_row['stop_time']
    seg_start_time = seg_end_time - segment_duration

    # Subset relevant rows
    seg_subset = df_sub[
        (df_sub['segment'] == segment_id) &
        (df_sub['time'] >= seg_start_time) &
        (df_sub['time'] <= seg_end_time)
    ].copy()  # copy to avoid SettingWithCopyWarning

    # Create time bins and assign bin index
    time_bins = np.arange(seg_start_time, seg_end_time, bin_width)
    seg_subset['bin'] = np.digitize(seg_subset['time'], time_bins) - 1
    seg_subset['seg_start_time'] = seg_start_time
    seg_subset['seg_end_time'] = seg_end_time
    seg_subset['seg_duration'] = seg_end_time - seg_start_time

    all_seg_subset.append(seg_subset)

# Concatenate all processed segments
all_seg_df_long = pd.concat(all_seg_subset, ignore_index=True)
all_seg_df_long.sort_values(by=['segment', 'bin'], inplace=True)
all_seg_df = all_seg_df_long.groupby(['segment', 'bin']).median().reset_index(drop=False)


In [ ]:
pn.retrieve_or_make_monkey_data()
behav_segment_df = all_seg_df[['segment', 'seg_start_time', 'seg_end_time', 'seg_duration']].drop_duplicates()
pn.prepare_spikes_for_gpfa(behav_segment_df, align_at_beginning=True, bin_width=bin_width)

Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new


In [49]:
pn.spiketrains 

[[SpikeTrain containing 10 spikes; units s; datatype float64 
  time: 0.0 s to 2.2 s,
  SpikeTrain containing 13 spikes; units s; datatype float64 
  time: 0.0 s to 2.2 s,
  SpikeTrain containing 10 spikes; units s; datatype float64 
  time: 0.0 s to 2.2 s,
  SpikeTrain containing 52 spikes; units s; datatype float64 
  time: 0.0 s to 2.2 s,
  SpikeTrain containing 3 spikes; units s; datatype float64 
  time: 0.0 s to 2.2 s,
  SpikeTrain containing 7 spikes; units s; datatype float64 
  time: 0.0 s to 2.2 s,
  SpikeTrain containing 2 spikes; units s; datatype float64 
  time: 0.0 s to 2.2 s,
  SpikeTrain containing 3 spikes; units s; datatype float64 
  time: 0.0 s to 2.2 s,
  SpikeTrain containing 3 spikes; units s; datatype float64 
  time: 0.0 s to 2.2 s,
  SpikeTrain containing 10 spikes; units s; datatype float64 
  time: 0.0 s to 2.2 s,
  SpikeTrain containing 16 spikes; units s; datatype float64 
  time: 0.0 s to 2.2 s,
  SpikeTrain containing 39 spikes; units s; datatype float6

In [65]:
import quantities as pq

In [66]:
pn.bin_width = 0.2
pn.bin_width_w_unit = pn.bin_width * pq.s

In [67]:
pn.get_gpfa_traj(latent_dimensionality=2, exists_ok=False)

Computing GPFA trajectories with 2 dimensions...


/opt/miniconda3/envs/ff_venv2/lib/python3.11/site-packages/elephant/gpfa/gpfa_util.py:131: UserWarning: trial corresponding to index 0 shorter than one segLength...skipping
  warnings.warn(
/opt/miniconda3/envs/ff_venv2/lib/python3.11/site-packages/elephant/gpfa/gpfa_util.py:131: UserWarning: trial corresponding to index 1 shorter than one segLength...skipping
  warnings.warn(
/opt/miniconda3/envs/ff_venv2/lib/python3.11/site-packages/elephant/gpfa/gpfa_util.py:131: UserWarning: trial corresponding to index 2 shorter than one segLength...skipping
  warnings.warn(
/opt/miniconda3/envs/ff_venv2/lib/python3.11/site-packages/elephant/gpfa/gpfa_util.py:131: UserWarning: trial corresponding to index 3 shorter than one segLength...skipping
  warnings.warn(
/opt/miniconda3/envs/ff_venv2/lib/python3.11/site-packages/elephant/gpfa/gpfa_util.py:131: UserWarning: trial corresponding to index 4 shorter than one segLength...skipping
  warnings.warn(
/opt/miniconda3/envs/ff_venv2/lib/python3.11/site-

Initializing parameters using factor analysis...

Fitting GPFA model...
Saved GPFA trajectories to all_monkey_data/decoding_targets/monkey_Schro/data_0416/gpfa_neural_aligned_segStart_d2.pkl


In [68]:
pn.trajectories[0].shape

(2, 11)

In [ ]:
# the saving directory is definitely wrong haha
Saved GPFA trajectories to all_monkey_data/decoding_targets/monkey_Schro/data_0416/gpfa_neural_aligned_segStart_d2.pkl

## get the corresponding spikes

# get GPFA spike data

In [ ]:
pn.behav_segment_df = pn.planning_data_by_point.drop_duplicates(subset=['segment'])
pn.behav_segment_df
pn.prepare_spikes_for_gpfa(align_at_beginning=False, behav_segment_df=pn.behav_segment_df)

# Appendix

## check NA

In [8]:
general_utils.check_na_in_df(pn.planning_data_by_bin)


NA Values Analysis for DataFrame (14,484 rows)

Number of rows with at least one NA value: 521

Columns with NA values:
------------------------------------------------------------
cur_cntr_arc_curv                             419 (   2.9%)
nxt_opt_arc_curv                              109 (   0.8%)
nxt_cntr_arc_curv                             109 (   0.8%)
cur_opt_arc_curv                              419 (   2.9%)
cur_opt_arc_end_heading                       418 (   2.9%)
cur_cntr_arc_end_heading                      418 (   2.9%)
angle_opt_arc_from_cur_end_to_nxt             520 (   3.6%)
angle_cntr_arc_from_cur_end_to_nxt            520 (   3.6%)
------------------------------------------------------------


(       point_index          bin  cur_opt_arc_dheading  cur_cntr_arc_curv  \
 244           1798   1494.00000               1.25507            0.00481   
 245           1799   1495.00000               1.25507            0.00481   
 246           1800   1496.00000               1.25507            0.00481   
 248           1802   1497.00000               1.25507            0.00481   
 249           1803   1498.00000               1.25507            0.00481   
 ...            ...          ...                   ...                ...   
 17526       144573 119682.00000               1.57080                NaN   
 17527       144574 119683.00000               1.57080                NaN   
 17528       144575 119684.00000               1.57080                NaN   
 17529       144576 119685.00000               1.57080                NaN   
 17531       144578 119686.00000               1.57080                NaN   
 
        segment_duration  nxt_ff_distance  nxt_opt_arc_dheading  \
 244   

In [5]:
pn.planning_data_by_point.shape

(17519, 47)

In [26]:
general_utils.check_na_in_df(pn.planning_data_by_bin)


NA Values Analysis for DataFrame (14,488 rows)

Number of rows with at least one NA value: 524

Columns with NA values:
------------------------------------------------------------
cur_cntr_arc_curv                             423 (   2.9%)
nxt_opt_arc_curv                              108 (   0.7%)
nxt_cntr_arc_curv                             108 (   0.7%)
cur_opt_arc_curv                              423 (   2.9%)
cur_opt_arc_end_heading                       422 (   2.9%)
cur_cntr_arc_end_heading                      422 (   2.9%)
angle_opt_arc_from_cur_end_to_nxt             523 (   3.6%)
angle_cntr_arc_from_cur_end_to_nxt            523 (   3.6%)
------------------------------------------------------------


(       point_index          bin  cur_opt_arc_dheading  cur_cntr_arc_curv  \
 244           1798   1494.00000               1.25507            0.00481   
 245           1799   1495.00000               1.25507            0.00481   
 246           1800   1496.00000               1.25507            0.00481   
 247           1801   1497.00000               1.25507            0.00481   
 249           1803   1498.00000               1.25507            0.00481   
 ...            ...          ...                   ...                ...   
 17526       144573 119682.00000               1.57080                NaN   
 17527       144574 119683.00000               1.57080                NaN   
 17528       144575 119684.00000               1.57080                NaN   
 17529       144576 119685.00000               1.57080                NaN   
 17530       144577 119686.00000               1.57080                NaN   
 
        segment_duration  nxt_ff_distance  nxt_opt_arc_dheading  \
 244   

In [10]:
general_utils.check_na_in_df(pn.planning_data_by_point)


NA Values Analysis for DataFrame (17,519 rows)

Number of rows with at least one NA value: 630

Columns with NA values:
------------------------------------------------------------
cur_opt_arc_dheading                          509 (   2.9%)
cur_cntr_arc_curv                             509 (   2.9%)
nxt_opt_arc_dheading                          130 (   0.7%)
nxt_opt_arc_curv                              130 (   0.7%)
nxt_cntr_arc_curv                             130 (   0.7%)
cur_opt_arc_curv                              509 (   2.9%)
cur_opt_arc_end_heading                       508 (   2.9%)
cur_cntr_arc_end_heading                      508 (   2.9%)
angle_opt_arc_from_cur_end_to_nxt             629 (   3.6%)
angle_cntr_arc_from_cur_end_to_nxt            629 (   3.6%)
------------------------------------------------------------


(       cur_opt_arc_dheading  cur_cntr_arc_curv  segment_duration  \
 244                 1.25507            0.00481           2.00848   
 245                 1.25507            0.00481           2.00848   
 246                 1.25507            0.00481           2.00848   
 247                 1.25507            0.00481           2.00848   
 248                 1.25507            0.00481           2.00848   
 ...                     ...                ...               ...   
 17514                   NaN                NaN           2.00854   
 17515                   NaN                NaN           2.00854   
 17516                   NaN                NaN           2.00854   
 17517                   NaN                NaN           2.00854   
 17518                   NaN                NaN           2.00854   
 
        nxt_ff_distance  nxt_opt_arc_dheading  nxt_opt_arc_curv  \
 244          637.86090                   NaN               NaN   
 245          637.86090             

In [23]:
general_utils.check_na_in_df(pn.ctrl_data_inst.both_ff_across_time_df)


NA Values Analysis for DataFrame (8,093 rows)

Number of rows with at least one NA value: 358

Columns with NA values:
------------------------------------------------------------
nxt_cntr_arc_curv                             130 (   1.6%)
cur_cntr_arc_curv                             237 (   2.9%)
nxt_opt_arc_dheading                          130 (   1.6%)
cur_opt_arc_curv                              237 (   2.9%)
nxt_opt_arc_curv                              130 (   1.6%)
cur_opt_arc_dheading                          237 (   2.9%)
cur_opt_arc_end_heading                       237 (   2.9%)
cur_cntr_arc_end_heading                      237 (   2.9%)
angle_opt_arc_from_cur_end_to_nxt             358 (   4.4%)
angle_cntr_arc_from_cur_end_to_nxt            358 (   4.4%)
------------------------------------------------------------


(      segment  nxt_cntr_arc_curv  time_rel_to_stop  cur_cntr_arc_curv  \
 0           0                NaN           2.00848            0.00481   
 1           0                NaN           1.99186            0.00481   
 2           0                NaN           1.97526            0.00481   
 3           0                NaN           1.95871            0.00481   
 4           0                NaN           1.94203            0.00481   
 ...       ...                ...               ...                ...   
 8088       71            0.00032           0.06642                NaN   
 8089       71            0.00032           0.04981                NaN   
 8090       71            0.00032           0.03328                NaN   
 8091       71            0.00031           0.01657                NaN   
 8092       71            0.00031           0.00000                NaN   
 
       nxt_opt_arc_dheading  target_index  cur_opt_arc_curv       time  \
 0                      NaN         